In [1]:
import pyspark.pandas as ps

StatementMeta(sparkctprdnc, 5, 1, Finished, Available)

'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.


### Combine UPC files for Silver layer

In [2]:
path = "abfss://fscustransdevnorthcent@stcustransdevnorthcent.dfs.core.windows.net/bronze/UPC/*/*/*/*.json"
sdfUPCBronzeData_raw = spark.read.option("Multiline","true").json(path)
sdfUPCBronzeData_raw.createOrReplaceTempView("vwUPCBronzeData_Raw")

StatementMeta(sparkctprdnc, 5, 2, Finished, Available)

In [3]:
sqlQuery = """
WITH cteUPCBronzeData_Raw AS (
    SELECT explode(body) as rowdata 
    FROM vwUPCBronzeData_Raw
)

SELECT 
     rowdata["Policy Number"] AS PolicyNumber
    ,rowdata["Insured"] AS Insured
    ,rowdata["Description "] AS Description
    ,rowdata["Date Sent"] AS DateSent        
FROM cteUPCBronzeData_Raw"""

StatementMeta(sparkctprdnc, 5, 3, Finished, Available)

In [4]:
sdfUPCBronzeData = spark.sql(sqlQuery)

StatementMeta(sparkctprdnc, 5, 4, Finished, Available)

In [9]:
path = 'abfss://fscustransdevnorthcent@stcustransdevnorthcent.dfs.core.windows.net/silver/upcsilver_raw'
sdfUPCBronzeData.write.format("delta").mode("overwrite").save(path)


StatementMeta(sparkctprdnc, 5, 9, Finished, Available)